In [10]:
import requests 
import pandas as pd
date = pd.to_datetime("2021/6/23")
url = "https://www.twse.com.tw/exchangeReport/MI_INDEX"
payloads = {
    "response":"html",
    "date": date.strftime("&Y&m&d"),
    "type":"ALLBUT0999"
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
}
response = requests.get(url, params = payloads, headers = headers)
response.text

'<!doctype html>\n<html lang="zh">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <title>TWSE 臺灣證券交易所 - 報表</title>\n  <style>\n    table { border-collapse: collapse; margin: 1em 0; }\n    table, th, td { border: 1px solid lightgray; }\n    table thead th, table thead td { text-align: center; background-color: #eee; }\n    table thead div { font-size: 20px; padding: 10px; }\n    table tbody tr:nth-child(even){ background-color:#f8f8f8; }\n    \n    td { padding: 4px; }\n    td>p { margin: 0; text-align: center; }\n    div.notes { margin: 2em 0 4em 0; line-height: 1.5em; }\n\n    table tbody td { text-align: right; }\n    table tbody td:first-child { text-align: left; }\n\n  </style>\n</head>\n<body>\n<div>\n    \n        <h2></h2>\n        \n            <table>\n            <thead>\n                <tr>\n                    \n                        <th colspan=\'6\'>110年06月23日 價格指數(臺灣證券交易所)</th>

In [11]:
df = pd.read_html(response.text)[-1]

df.columns =df.columns.get_level_values(2)

df["漲跌價差"]=df["漲跌價差"].where(df['漲跌(+/-)']!="-", - df["漲跌價差"])

df.drop(["證券名稱", "漲跌(+/-)"], inplace=True, axis=1)

df["日期"] = pd.to_datetime(date)

df = df.set_index(["證券代號", "日期"])

df = df.apply(pd.to_numeric, errors="coerce")

df.drop(df[df["收盤價"].isnull()].index, inplace=True)

df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']

df['股價振幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100

df

,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價振幅
證券代號,日期,,,,,,,,,,,,,,,
0050,2021-06-23,6517135,5282,887399669,135.00,137.15,134.80,137.05,2.20,137.00,53,137.05,1,0.00,134.85,1.742677
0051,2021-06-23,118429,47,6735373,56.95,56.95,56.50,56.95,0.45,56.95,34,57.00,12,0.00,56.50,0.796460
0052,2021-06-23,1016662,425,126361688,123.40,125.10,122.60,125.00,3.00,124.60,31,125.00,9,0.00,122.00,2.049180
0053,2021-06-23,7887,21,517320,65.20,66.00,65.10,66.00,1.45,66.05,1,66.55,1,0.00,64.55,1.394268
0054,2021-06-23,8041,7,251493,31.18,31.47,31.18,31.47,0.44,31.26,24,31.67,19,0.00,31.03,0.934579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2021-06-23,182696,110,3922774,21.45,21.55,21.40,21.50,0.00,21.45,15,21.50,9,29.05,21.50,0.697674
9945,2021-06-23,3292900,1917,177655987,53.80,54.10,53.70,54.00,0.20,54.00,5,54.10,65,6.63,53.80,0.743494
9946,2021-06-23,251682,112,3735473,14.60,14.95,14.55,14.85,0.25,14.85,1,14.90,5,92.81,14.60,2.739726


In [15]:
df[df['成交股數']>6]
df.sort_values(by=['成交筆數'], ascending=False).head(10)

,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價振幅
證券代號,日期,,,,,,,,,,,,,,,
2615,2021-06-23,270589534,145340,65964784486,246.00,262.00,226.0,226.00,-25.00,NaN,0,226.00,4196,19.33,251.00,14.342629
2603,2021-06-23,174223591,123163,24909763848,148.00,150.50,138.5,138.50,-15.00,NaN,0,138.50,4650,11.88,153.50,7.817590
2601,2021-06-23,361275279,104866,6813961126,20.50,20.65,17.3,19.00,-0.10,18.95,2,19.00,811,0.00,19.10,17.539267
2609,2021-06-23,135256310,100084,19332606158,146.00,150.00,139.5,139.50,-15.00,NaN,0,139.50,21706,12.46,154.50,6.796117
2014,2021-06-23,242804514,95916,12466006227,53.00,54.80,48.2,48.25,-3.45,48.20,1616,48.25,182,29.42,51.70,12.765957
3481,2021-06-23,404145338,77415,8643416548,21.15,21.70,20.9,21.50,0.65,21.45,1945,21.50,2567,11.94,20.85,3.836930
2409,2021-06-23,311874504,70203,7490417637,23.95,24.35,23.5,24.20,0.70,24.15,902,24.20,2034,11.36,23.50,3.617021
2637,2021-06-23,136632259,69602,12837099404,96.90,101.00,89.8,89.80,-9.90,NaN,0,89.80,5558,59.47,99.70,11.233701
2610,2021-06-23,224575419,64924,4391343182,20.15,20.40,19.1,19.50,-0.40,19.50,563,19.55,880,36.79,19.90,6.532663
